# Data Analysis over the Clusters
## Loading libraries

In [1]:
import numpy as np
import cygnus_lib as cy
import toolslib as tl
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

from iDBSCAN import iDBSCAN
from sklearn.cluster import DBSCAN


from time import time
from ast import literal_eval
from math import degrees
from math import radians
from toolslib import colorbar


## font definition
%matplotlib inline
plt.rcParams['font.family'] = 'sans-serif'
plt.rcParams['font.sans-serif'] = ['Bitstream Vera Sans']
plt.rcParams['font.serif'] = ['Bitstream Vera Sans']
cy.set_atlas_style('large')

sns.set_context('poster')
sns.set_style('white')
sns.set_color_codes()
plot_kwds = {'alpha' : 0.5, 's' : 30, 'linewidths':0}

## Variables definition

In [2]:
x_resolution = y_resolution = 2048
rescale      = 512
scale        = int(x_resolution/rescale)
pixelscale   = 55e-3             #55e-3 for Orange ------ 0.125 for lemonn mm/pixel

## Loading data

List contain:

[0 - Run number, 1 - Image Number, 2 - Tag of the cluster, 3 - Pixel X position, 4 - Pixel Y position, 5 - Light in the pixel, 6 - Pedestal in the pixel]

In [3]:
tic=time()
#------------------- Loading File ------------------------------#
directory = ("./data/")          # Directory of the output file
filename  = ("toEff_L")            # Name of the output file
extension = (".csv")             # Extension of the output file
dataout   = directory + filename + extension # Full path of the output file

dt        = {'Run': np.int64, 'Image': np.int64, 'Tag': np.object, 'X': np.object, 
             'Y': np.object, 'Light': np.object, 'Pedestal': np.object}
colhead   = ["Run","Image","Tag","X","Y","Light","Pedestal"]
dfl       = pd.read_csv(dataout,dtype=dt)

dfl.loc[:,'X']        = dfl.loc[:,'X'].apply(literal_eval)
dfl.loc[:,'Y']        = dfl.loc[:,'Y'].apply(literal_eval)
dfl.loc[:,'Light']    = dfl.loc[:,'Light'].apply(literal_eval)
dfl.loc[:,'Pedestal'] = dfl.loc[:,'Pedestal'].apply(literal_eval)

In [4]:
#------------------- Loading File ------------------------------#
directory = ("./data/")          # Directory of the output file
filename  = ("toEff_M")            # Name of the output file
extension = (".csv")             # Extension of the output file
dataout   = directory + filename + extension # Full path of the output file

dt        = {'Run': np.int64, 'Image': np.int64, 'Tag': np.object, 'X': np.object, 
             'Y': np.object, 'Light': np.object, 'Pedestal': np.object}
colhead   = ["Run","Image","Tag","X","Y","Light","Pedestal"]
dfm       = pd.read_csv(dataout,dtype=dt)

dfm.loc[:,'X']        = dfm.loc[:,'X'].apply(literal_eval)
dfm.loc[:,'Y']        = dfm.loc[:,'Y'].apply(literal_eval)
dfm.loc[:,'Light']    = dfm.loc[:,'Light'].apply(literal_eval)
dfm.loc[:,'Pedestal'] = dfm.loc[:,'Pedestal'].apply(literal_eval)

In [5]:
#------------------- Loading File ------------------------------#
directory = ("./data/")          # Directory of the output file
filename  = ("toEff_S")            # Name of the output file
extension = (".csv")             # Extension of the output file
dataout   = directory + filename + extension # Full path of the output file

dt        = {'Run': np.int64, 'Image': np.int64, 'Tag': np.object, 'X': np.object, 
             'Y': np.object, 'Light': np.object, 'Pedestal': np.object}
colhead   = ["Run","Image","Tag","X","Y","Light","Pedestal"]
dfs       = pd.read_csv(dataout,dtype=dt)

dfs.loc[:,'X']        = dfs.loc[:,'X'].apply(literal_eval)
dfs.loc[:,'Y']        = dfs.loc[:,'Y'].apply(literal_eval)
dfs.loc[:,'Light']    = dfs.loc[:,'Light'].apply(literal_eval)
dfs.loc[:,'Pedestal'] = dfs.loc[:,'Pedestal'].apply(literal_eval)

toc = time()
print("Loading time: %.2f" % ((toc-tic)/60))

Loading time: 0.18


In [6]:
frames = [dfl, dfm, dfs]
df = pd.concat(frames, ignore_index = True)

del frames

In [ ]:
def pointInsideCircle(Px, Py, Cx = 1024, Cy = 1024, R = 900):
    Hx = np.zeros(4,dtype=float)
    Hy = np.zeros(4,dtype=float)
    
    Hx[0] = np.min(Px)
    Hy[0] = Py[np.argmin(Px)]
    
    Hx[1] = np.max(Px)
    Hy[1] = Py[np.argmax(Px)]
      
    Hx[2] = np.min(Py)
    Hy[2] = Px[np.argmin(Py)]
    
    Hx[3] = np.max(Py)
    Hy[3] = Px[np.argmax(Py)]
    
    out = np.ones(4, dtype=bool)
    for i in range(0,len(Hx)):
        
        teste = np.sqrt((Hx[i]-Cx)**2 + (Hy[i]-Cy)**2)
        if teste < R:
            out[i] = True
        else:
            out[i] = False

    result = np.all(out)
    
    return result

In [ ]:
# List to salve all simulated images and informations
clutruth   = []
truth      = []
tag        = []
pedestal   = 99

flag_angle = False
flag_bkg   = False
flag_plt   = False

runI  = [494]
nRi   = 0

nfigures   = 10

if flag_bkg == False:
    fileoutm = ("./data/run%d_mean.h5" % (runI[nRi]))
    m_image = cy.read_image_h5(fileoutm)
    fileouts = ("./data/run%d_sigma.h5" % (runI[nRi]))
    s_image = cy.read_image_h5(fileouts)

for jj in range(0,nfigures):
    #Background
    
    if flag_bkg == True:
        dataSelection = 'LAB'
        cleImg = [4,5,7,9,15]
        iTr   = cleImg[np.random.randint(0, len(cleImg))]
        imageOri = cy.swift_read_image_h5(cy.imageFile2FullPathCygnus(dataSelection, runI[0], iTr)) 
        titletext = "I%d Run%d + one random long track" % (iTr, runI[0])
        print("Using I%d Run%d as background noise" % (iTr, runI[0]))
        
    else:
        # To remove any NaN
        s_image[np.isnan(s_image)] = np.mean(s_image[~np.isnan(s_image)])
        m_image[np.isnan(m_image)] = np.mean(m_image[~np.isnan(m_image)])

        m_image[m_image > 101] = np.mean(m_image[m_image < 101])
        s_image[s_image > 4]   = np.mean(s_image[s_image < 4])

        imageOri = np.random.normal(m_image,s_image,[2048,2048])
        titletext = "Inserting background noise"
        print("Create background noise from mean + sigma of the run")
            

    #matrix = np.zeros([y_resolution,x_resolution],dtype=int) # Background
    image    = np.copy(imageOri)

    it       = -1 
    
    nSmall   = 0
    nMedium  = 0
    nLong    = 3
    
    fname  = np.str(nLong) + "L" + np.str(nMedium) + "M" + np.str(nSmall) + "S"

    #rd       = np.concatenate([np.random.randint(0, dfl.shape[0],nLong),np.random.randint(11, 15 + 1,nMedium),
    #                           np.random.randint(16, 20 + 1,nSmall)])
    
    rd       = np.concatenate([np.random.randint(0, dfl.shape[0],nLong),
                               np.random.randint(dfl.shape[0], dfl.shape[0] + dfm.shape[0],nMedium),
                               np.random.randint(dfl.shape[0] + dfm.shape[0],
                                                 dfl.shape[0] + dfm.shape[0] + dfs.shape[0],nSmall)])
    
    for kk in range(0,nLong):
        tag.append('l')
    for kk in range(0,nMedium):
        tag.append('m')
    for kk in range(0,nSmall):
        tag.append('s')
        
    for ind in rd:
        it += 1
        
        if flag_angle == True:
            angle             = radians(np.random.randint(0, 360 + 1))
            #newX,newY         = tl.rotate(df.X[ind][0],df.Y[ind][0],df.X[ind],df.Y[ind],angle)
            newX,newY         = tl.rotate(df.X[ind][0],df.Y[ind][0],df.X[ind],df.Y[ind],angle)
            #newX,newY         = tl.rotate(0,0,df.X[ind],df.Y[ind],angle)
            newX              = np.round(newX).astype(int)
            newY              = np.round(newY).astype(int)

            if np.min(newX) <= 0:
                newX = newX +np.abs(np.min(newX))
            if np.max(newX) >= 2047:
                newX = newX - (np.abs(np.max(newX)) - 2047)

            if np.min(newY) <= 0:
                newY = newY +np.abs(np.min(newY))
            if np.max(newY) >= 2047:
                newY = newY - (np.abs(np.max(newY)) - 2047)
        else:
            newX = np.array(df.X[ind])
            newY = np.array(df.Y[ind])

        flag_inside = False
        while (flag_inside == False):
            addX = [np.random.randint(0, 2047 + 1 - np.max(newX)), -1*np.random.randint(0,np.min(newX) + 1)]
            addY = [np.random.randint(0, 2047 + 1 - np.max(newY)), -1*np.random.randint(0,np.min(newY) + 1)]
            mm = np.random.randint(0, 2)

            newX              = newX+addX[mm]
            newY              = newY+addY[mm]

            flag_inside = pointInsideCircle(newX, newY, Cx = 1024, Cy = 1024, R = 900)
        
        Lp                = df[colhead[5]][ind]
        image[newY,newX]  = (np.array(Lp)-pedestal) + image[newY,newX]
        
        infc = []
        infc.append(jj)
        infc.append(newX)
        infc.append(newY)
        infc.append(Lp)
        infc.append(tag[it])
        clutruth.append(infc)

    if flag_plt == True:
        fig = plt.figure(figsize=(15,15))
        ax  = plt.gca()

        iax = ax.imshow(image, cmap = "viridis", vmin = 85, vmax = 130)
        ax.set_xlim(0,2047)
        ax.set_ylim(2047,0)
        ax.set_title(titletext)
        colorbar(iax)
        plt.show(block=False)
        plt.close()

    print("Index Long track: %d" % (rd[0]))   
    
    inf = []
    inf.append(image)
    inf.append(jj)
    truth.append(inf)

Create background noise from mean + sigma of the run
Index Long track: 12
Create background noise from mean + sigma of the run
Index Long track: 17
Create background noise from mean + sigma of the run
Index Long track: 16
Create background noise from mean + sigma of the run
Index Long track: 19
Create background noise from mean + sigma of the run
Index Long track: 10
Create background noise from mean + sigma of the run
Index Long track: 0
Create background noise from mean + sigma of the run
Index Long track: 0
Create background noise from mean + sigma of the run
Index Long track: 11
Create background noise from mean + sigma of the run
Index Long track: 8
Create background noise from mean + sigma of the run
Index Long track: 10


# i2DBSCAN vs DBSCAN

## Setup

In [ ]:
################ analysis cards ################################
nsigma       = 1         # numero di sigma sopra il piedistallo
cimax        = 200       # valori del cut sull'imagine
rescale      = 512       # binnagio finale immagine (deve essre un sottomultipli della 2**2 risluzione di partenza)
minClose     = 2         # minimum cluser size (rebinne image)
eps          = 5         # maximum distance for the cluster point
maxClose     = 30000     # massima dimesione del clustr evita le scriche      
Cmethod      = 'idbsc'    #'hdbs' # 'nccs' # 'dbsc' # 'idbsc'
max_image_to_read = 60  # 0 all
############### Inzializzazione varibili e costanti #################
scale        = int(x_resolution/rescale)



iterative    = 1         # number of iterations for the IDBSC
vector_eps = [2.26, 3.5, 2.8, 6]
vector_min_samples = [2, 28, 4, 2]
cuts = [775, 150]


# MAIN LOOP ON 

dataNaive    = [] # output data
datai2DB     = [] # output data

In [ ]:
#
# load pedestal value generated by runs-pedestals.ipynb script
# 
try:
    fileoutm = ("./data/run%d_mean.h5" % (runI[nRi]))
    m_image = cy.read_image_h5(fileoutm)
    PedOverMax = m_image[m_image > cimax].size
    print ("Pedestal mean: %.2f, sigma: %.2f, over th. (%d) %d" % 
       (m_image[m_image<cimax].mean(), 
        np.sqrt(m_image[m_image<cimax].var()), cimax,
        (m_image>cimax).sum()))
except:
    print ("No Pedestal file for run %s, run script runs-pedestals.ipynb" % runI[nRi])
    print ("STOP")
try: 
    fileouts = ("./data/run%d_sigma.h5" % (runI[nRi]))
    s_image = cy.read_image_h5(fileouts)
    print ("Sigma mean: %.2f, sigma: %.2f, over th. (50) %d" % 
   (s_image[s_image<50].mean(), 
    np.sqrt(s_image[s_image<50].var()), 
    (s_image>50).sum()))
except:
    print ("No Sigma file for run %s, run script runs-pedestals.ipynb" % runI[nRi])
    print ("STOP")

#
# Run by run init 
#
th_image      = np.round(m_image + nsigma*s_image) # verficare con il np.round.... np.ceil
th_image[:,:] = 102 # per imostare tutto a 101

TrOk   = 0
dCloseT= 0

Pedestal mean: 99.63, sigma: 2.02, over th. (200) 1955
Sigma mean: 3.23, sigma: 2.23, over th. (50) 2208


/usr/lib/python3.6/site-packages/ipykernel_launcher.py:19: RuntimeWarning: invalid value encountered in less
/usr/lib/python3.6/site-packages/ipykernel_launcher.py:20: RuntimeWarning: invalid value encountered in less
/usr/lib/python3.6/site-packages/ipykernel_launcher.py:21: RuntimeWarning: invalid value encountered in greater


In [ ]:
# Truth
tnumPixT = 0
tnumCluT = len(clutruth)
teneCluT = 0

for it in range(0,len(truth)):
    tnumPixT = tnumPixT + np.size(clutruth[it][1])
    teneCluT = teneCluT + np.sum(clutruth[it][3])

print("Total number of clusters: %d" % (tnumCluT))
print("Total ligth on all clusters: %d" % (teneCluT))
print("Total number of pixels on all clusters: %d" % (tnumPixT))

Total number of clusters: 30
Total ligth on all clusters: 30735165
Total number of pixels on all clusters: 265120


In [ ]:
flag_naive = False

vectoreps = np.concatenate([list(np.linspace(0.1,4,30)),list(np.linspace(5,10,5)),list(np.linspace(10,50,10))])
vectorminp = np.round(np.concatenate([list(np.linspace(1,10,10)),list(np.linspace(15,150,10)),list(np.linspace(160,240,20))]))
                      
tamX = len(vectoreps)
tamY = len(vectorminp)

info = []
tnumPix = np.zeros([tamX,tamY],dtype=int)
tnumClu = np.zeros([tamX,tamY],dtype=int)
teneClu = np.zeros([tamX,tamY],dtype=int)

aux0 = -1

for eps in vectoreps:
    aux0 +=1
    aux1 = -1
    for minp in vectorminp:
        aux1 +=1

        for it in range(0,len(truth)):
            edges_image     = (truth[it][0] > th_image) & (truth[it][0] < cimax)
            rebin_image     = cy.rebin(truth[it][0], (rescale, rescale))
            rebin_th_image  = cy.rebin(th_image, (rescale, rescale))
            edges           = (rebin_image > rebin_th_image) & (rebin_image < cimax)         
            points          = np.array(np.nonzero(edges)).T.astype(float)

            #clusters = iDBSCAN(iterative).fit(points)
            vector_eps[1] = eps
            vector_min_samples[1] = minp
            
            clusters = iDBSCAN(iterative = 1, vector_eps = vector_eps, vector_min_samples = vector_min_samples, cuts = cuts).fit(points)


            n_clusters_  = len(set(clusters.labels_)) - (1 if -1 in clusters.labels_ else 0)

            tnumClu[aux0,aux1] = tnumClu[aux0,aux1] + n_clusters_


            for Ci in range (0, n_clusters_): # number of clusters found in the image
                cluvar   = [] # cluster info
                #print ("Salving cluster %d from %d" % (Ci,n_clusters_))
                Xi, Yi, Is, Ib, tag = tl.cluInfo(clusters,points,Ci,truth[it][0],th_image,scale) # extract cluster information
                # cluvar = [Run, Imag, Tag, X position, Y position, Light signal, Light pedestal]
                cluvar.append(runI[nRi])
                cluvar.append(clutruth[it][4])
                cluvar.append(tag)
                cluvar.append(Xi)
                cluvar.append(Yi)
                cluvar.append(Is)
                cluvar.append(Ib)
                datai2DB.append(cluvar) # save the cluster info in a List
                tnumPix[aux0,aux1] = tnumPix[aux0,aux1] + np.size(Xi)
                teneClu[aux0,aux1] = teneClu[aux0,aux1] + np.sum(Is)


            if flag_naive == True:

                naive    = iDBSCAN(iterative = 0, vector_eps = [2.6, 3.5, 2.8, 6], 
                                   vector_min_samples = [2, 30, 6, 2], cuts = cuts).fit(points)

                n_clusters_n = len(set(naive.labels_)) - (1 if -1 in naive.labels_ else 0)
                for Ci in range (0, n_clusters_n): # number of clusters found in the image
                    cluvar   = [] # cluster info
                    #print ("Salving cluster %d from %d" % (Ci,n_clusters_))
                    Xi, Yi, Is, Ib, tag = tl.cluInfo(naive,points,Ci,truth[it][0],th_image,scale) # extract cluster information
                    # cluvar = [Run, Imag, Tag, X position, Y position, Light signal, Light pedestal]
                    cluvar.append(runI[nRi])
                    cluvar.append(clutruth[it][4])
                    cluvar.append(tag)
                    cluvar.append(Xi)
                    cluvar.append(Yi)
                    cluvar.append(Is)
                    cluvar.append(Ib)
                    dataNaive.append(cluvar) # save the cluster info in a List

    print ("...Next EPS...%f%% \n" % (((aux0+1)/tamX)*100))

...Next EPS...2.222222% 

...Next EPS...4.444444% 

...Next EPS...6.666667% 

...Next EPS...8.888889% 

...Next EPS...11.111111% 

...Next EPS...13.333333% 

...Next EPS...15.555556% 

...Next EPS...17.777778% 

...Next EPS...20.000000% 

...Next EPS...22.222222% 

...Next EPS...24.444444% 

...Next EPS...26.666667% 

...Next EPS...28.888889% 

...Next EPS...31.111111% 

...Next EPS...33.333333% 

...Next EPS...35.555556% 

...Next EPS...37.777778% 

...Next EPS...40.000000% 

...Next EPS...42.222222% 

...Next EPS...44.444444% 

...Next EPS...46.666667% 

...Next EPS...48.888889% 

...Next EPS...51.111111% 

...Next EPS...53.333333% 

...Next EPS...55.555556% 

...Next EPS...57.777778% 

...Next EPS...60.000000% 

...Next EPS...62.222222% 

...Next EPS...64.444444% 

...Next EPS...66.666667% 

...Next EPS...68.888889% 

...Next EPS...71.111111% 

...Next EPS...73.333333% 

...Next EPS...75.555556% 

...Next EPS...77.777778% 

...Next EPS...80.000000% 

...Next EPS...82.222222% 

...Ne

In [ ]:
# Make data.
Xx = vectoreps
Yy = vectorminp
Xx, Yy = np.meshgrid(Xx, Yy)
Z = teneClu

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt

azim = 180+45
elev = 30
un = 1000000

fig = plt.figure(figsize = (12,8))
ax  = fig.add_subplot(111, projection='3d')

ax.scatter(Yy.T,Xx.T, Z/un, c = 'r', marker = 'o')

ax.set_xlabel('MinPoints',labelpad = 20, rotation=0)
ax.set_ylabel('Eps',labelpad = 20)
ax.set_zlabel('Total light in all clusters [M]', labelpad = 10, rotation = 90)
ax.set_ylim(0,80)
ax.set_xlim(0,300)
ax.set_zlim(bottom = 0)
ax.view_init(elev = elev, azim = azim)

plt.savefig('./images/3D_TotalLight_' + fname + '.pdf', format='pdf',bbox_inches = 'tight', pad_inches = 0)

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt

Z = tnumClu

azim = 180+45
elev = 30
un = 1000

fig = plt.figure(figsize = (12,8))
ax  = fig.add_subplot(111, projection='3d')

ax.scatter(Yy.T,Xx.T, Z, c = 'r', marker = 'o')

ax.set_xlabel('MinPoints',labelpad = 20, rotation=0)
ax.set_ylabel('Eps',labelpad = 20)
ax.set_zlabel('Total number of clusters', labelpad = 10, rotation = 90)
ax.set_ylim(0,80)
ax.set_xlim(0,300)
ax.set_zlim(bottom = 0)
ax.view_init(elev = elev, azim = azim)

plt.savefig('./images/3D_TotalClu_' + fname + '.pdf', format='pdf',bbox_inches = 'tight', pad_inches = 0)

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter
import matplotlib.ticker as mticker
#import numpy as np

Z = tnumPix

#pylab.rcParams['ytick.major.pad']='8'

fig = plt.figure(figsize=(20,20))
ax = fig.gca(projection='3d')

# Plot the surface.

surf = ax.plot_surface(Xx.T,Yy.T,Z,cmap='viridis')
ax.set_xlabel('EPS parameter',labelpad = 20,rotation=-17)
ax.set_ylabel('Min_Points parameter',labelpad = 20,rotation=90)

ax.set_zlabel('Number of clusters [log]',labelpad = 60,rotation=0)
ax.tick_params(axis='z', which='major', pad=30)

In [ ]:
columnsT = ["X","Y","Ls","Image","Tag"]
dftruth = pd.DataFrame(clutruth, columns = columnsT)

columns = ["Run","Image","Tag","X","Y","Light","Pedestal"]
dfi = pd.DataFrame(datai2DB, columns = columns)

if flag_naive == True:
    columns = ["Run","Image","Tag","X","Y","Light","Pedestal"]
    dfn = pd.DataFrame(dataNaive, columns = columns)

## Efficiency evaluation

## BACKUP

In [ ]:
def get_colorpalette(idb):
    palette = sns.color_palette('deep', np.unique(idb.labels_).max() + 1)
    colors = [palette[x] if x >= 0 else (0.0, 0.0, 0.0) for x in idb.labels_]
    return colors

In [ ]:
X = points
colorsI = get_colorpalette(clusters)
colorsN = get_colorpalette(naive)


## PLOT Noise Rejection
fig, ax = plt.subplots(1,2,figsize=(16, 8), sharex=True, sharey=True)
ax[0].invert_yaxis()

ax[0].scatter(X[:, 1], X[:, 0], c=colorsI, **plot_kwds)
ax[0].set_title("i2DBSCAN")

ax[1].scatter(X[:, 1], X[:, 0], c=colorsN, **plot_kwds)
ax[1].set_title("Naive DBSCAN")

plt.show()

In [ ]:
th_image      = np.round(m_image + nsigma*s_image) # verficare con il np.round.... np.ceil
th_image[:,:] = 102 # per imostare tutto a 101


edges_image     = (truth[it][0] > th_image) & (truth[it][0] < cimax)
rebin_image     = cy.rebin(truth[it][0], (rescale, rescale))
rebin_th_image  = cy.rebin(th_image, (rescale, rescale))
edges           = (rebin_image > rebin_th_image) & (rebin_image < cimax)         
points          = np.array(np.nonzero(edges)).T.astype(float)


fig = plt.figure(figsize=(7,7))

plt.scatter(points[:, 1], points[:, 0],**plot_kwds)
plt.xlim(0,511)
plt.ylim(511,0)

## Background

In [ ]:
fig = plt.figure(figsize=(15,15))
ax  = plt.gca()

iax = ax.imshow(image,cmap="viridis", vmin=85,vmax=130)
ax.set_xlim(np.min(newX),np.max(newX))
ax.set_ylim(np.max(newY),np.min(newY))
ax.set_title("I%d Run%d + one random long track" % (iTr, runI[0]))
colorbar(iax)
plt.show(block=False)
plt.close()

In [ ]:
# To remove any NaN
s_image[np.isnan(s_image)] = np.mean(s_image[~np.isnan(s_image)])
m_image[np.isnan(m_image)] = np.mean(m_image[~np.isnan(m_image)])

m_image[m_image > 101] = np.mean(m_image[m_image < 101])
s_image[s_image > 4] = np.mean(s_image[s_image < 4])


n_image = np.random.normal(m_image,s_image,[2048,2048])

In [ ]:
fig = plt.figure(figsize=(15,15))
ax  = plt.gca()

iax = ax.imshow(n_image,cmap="viridis", vmin=85,vmax=130)
ax.set_xlim(0,2047)
ax.set_ylim(2047,0)
ax.set_title("Background Image")
colorbar(iax)
plt.show(block=False)
plt.close()